In [2]:
import scraping_class
logfile = 'log.txt'## name your log file.
connector = scraping_class.Connector(logfile)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

In [3]:
import pandas as pd
import numpy as np
import html5lib
import sys
import pickle
from tqdm import tqdm_notebook

In [6]:
with open('df_final-COPY1.pkl', 'rb') as f:
    df = pickle.load(f)

## Defining function to look into statistics

In [7]:
def tlen(name):
    tlen = pd.DataFrame()
    tlen_id = []
    tlen_value = []
    tlen_share = []
    for i in df[name].unique():
        tlen_id.append(i)
        tlen_value.append(len(df.loc[df[name] == i]))
        
    for i in df[name].unique():
        tlen_share.append((len(df.loc[df[name] == i])/sum(tlen_value)*100))
        
    tlen_id.append('Total')
    tlen_value.append(sum(tlen_value))
    tlen_share.append(sum(tlen_share))
        
        
    tlen['Id'] = tlen_id
    tlen['Value'] = tlen_value
    tlen['Share %'] = tlen_share
        
    return(tlen.sort_values('Value',ascending=True))

In [8]:
def tlen_year(name,year):
    df1 = df[df['Year'] == year]
    tlen = pd.DataFrame()
    tlen_id = []
    tlen_value = []
    tlen_share = []
    for i in df[name].unique():
        tlen_id.append(i)
        tlen_value.append(len(df1.loc[df[name] == i]))
    
    for i in df[name].unique():
        tlen_share.append((len(df1.loc[df[name] == i])/sum(tlen_value)*100))
        
    tlen_id.append('Total')
    tlen_value.append(sum(tlen_value))
    tlen_share.append(sum(tlen_share))
        
    tlen['Id'] = tlen_id
    tlen['Value'] = tlen_value
    tlen['Share %'] = tlen_share
        
    return(tlen.sort_values('Value',ascending=True))

In [10]:
def tlen_byyear(name):
    
    tlen = pd.DataFrame()
    tlen_id = []
    tlen_1314 = []
    tlen_1415 = []
    tlen_1516 = []
    tlen_1617 = []
    tlen_1718 = []

    for i in df[name].unique():
        tlen_id.append(i)
        df1 = df[df['Year'] == '2013-2014']
        tlen_1314.append(len(df1.loc[df[name] == i]))
        df2 = df[df['Year'] == '2014-2015']
        tlen_1415.append(len(df2.loc[df[name] == i]))
        df3 = df[df['Year'] == '2015-2016']
        tlen_1516.append(len(df3.loc[df[name] == i]))
        df4 = df[df['Year'] == '2016-2017']
        tlen_1617.append(len(df4.loc[df[name] == i]))
        df5 = df[df['Year'] == '2017-2018']
        tlen_1718.append(len(df5.loc[df[name] == i]))
    
    
        
    tlen_id.append('Total')
    tlen_1314.append(sum(tlen_1314))
    tlen_1415.append(sum(tlen_1415))
    tlen_1516.append(sum(tlen_1516))
    tlen_1617.append(sum(tlen_1617))
    tlen_1718.append(sum(tlen_1718))
    
        
    tlen['Id'] = tlen_id
    tlen['2013-2014'] = tlen_1314
    tlen['2014-2015'] = tlen_1415
    tlen['2015-2016'] = tlen_1516
    tlen['2016-2017'] = tlen_1617
    tlen['2017-2018'] = tlen_1718
        
    return(tlen)

In [ ]:


df_nocen = df[df['Censorship form_edit'] == 'No external censorship']
df_cen = df[df['Censorship form_edit'] == 'External censorship']
print(pd.to_numeric(df_cen['Eksamensgennemsnit']).describe(), pd.to_numeric(df_nocen['Eksamensgennemsnit']).describe())



# Descriptive statistic - diving in to the weirds of the data
## Years
It is seen from `tlen('year')` that there a many fewer eksamns in the study year 2013-2014 than the rest. By looking at `tlen_year('Termin','2013-2014')` it is seen that only the summer eksamns are in this year. By going to the grade web-page it is seen that it only tracks data to summer 14, and therefore eksamns from winter 13 not included.

It is seen that apparently 2015-2016 has around 500 more eksamns than both the year before and the year after. Most of them are from HUM as seen in `tlen_byyear('Fakultet')`. F.eks. TORS has 150 in 2015-2016 and suddenly 1 in 2016-2017. It seems like 2015-2016 as seen by `tlen_byyear('Institut')` maybe there where an extra focus on doing statistics in 2016-2017. This hypothesis is backed up by `tlen_byyear('Censorship form_edit')` where the rise is shown in nan. I.E the courses where the info on the exam is limited.

An Interesting fact from `tlen_byyear('Censorship form_edit')` is that no external is rising whil external is falling.

## Output:

#### Table. Disribution of year and Fakultet

In [16]:
print(tlen_byyear('Fakultet').to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                                       Id &  2013-2014 &  2014-2015 &  2015-2016 &  2016-2017 &  2017-2018 \\
\midrule
 Det Natur- og Biovidenskabelige Fakultet &        519 &       1161 &       1205 &       1185 &       1207 \\
      Det Samfundsvidenskabelige Fakultet &        225 &        456 &        534 &        497 &        513 \\
                Det Humanistiske Fakultet &        212 &        421 &        758 &        341 &        244 \\
      Det Sundhedsvidenskabelige Fakultet &        238 &        651 &        637 &        672 &        671 \\
                   Det Juridiske Fakultet &        117 &        269 &        310 &        288 &        280 \\
                  Det Teologiske Fakultet &         35 &         82 &         83 &         86 &         87 \\
                                    Total &       1346 &       3040 &       3527 &       3069 &       3002 \\
\bottomrule
\end{tabular}



#### List of categories we've made

In [58]:
cat_list = ['Fakultet', 'Eksamenstype', 'Year', 'Aid_edit', 'Marking scale_edit', 'Censorship form_edit', 'Duration_edit', 'Language_edit', 'Type of assessmet_edit']
di = {}
list_1 = []
for i in cat_list:
    list_1 = []
    for k in df[i].unique():
        list_1.append(k)
    di[i]=list_1
pd.DataFrame.from_dict(di, orient='index').T

,Fakultet,Eksamenstype,Year,Aid_edit,Marking scale_edit,Censorship form_edit,Duration_edit,Language_edit,Type of assessmet_edit
0,Det Natur- og Biovidenskabelige Fakultet,Ordinær eksamen,2014-2015,nan,7-point grading scale,No external censorship,1 block,English,Continuous assessment not exam
1,Det Samfundsvidenskabelige Fakultet,Reeksamen,2017-2018,All aids allowed,nan,External censorship,1 semester,Danish,Oral
2,Det Humanistiske Fakultet,None,2016-2017,Only certain aids allowed,passed/not passed,nan,Other/Not specified,Other,Written not under invigilation
3,Det Sundhedsvidenskabelige Fakultet,None,2015-2016,Written aids allowed,completed/not completed,None,2 blocks,None,Portfolio
4,Det Juridiske Fakultet,None,2013-2014,Without aids,Without assessment,None,None,None,Written under invigilation
5,Det Teologiske Fakultet,None,None,None,None,None,None,None,Practical
6,None,None,None,None,None,None,None,None,Course participation written
7,None,None,None,None,None,None,None,None,Course participation not written
8,None,None,None,None,None,None,None,None,Continuous assessment exam
9,None,None,None,None,None,None,None,None,Other


In [59]:
tlen_byyear('Censorship form_edit')

,Id,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018
0,No external censorship,603,1448,1598,1664,1814
1,External censorship,547,1245,1295,1120,993
2,nan,196,347,634,285,195
3,Total,1346,3040,3527,3069,3002


In [69]:
df_nocen = df[df['Censorship form_edit'] == 'No external censorship']
df_cen = df[df['Censorship form_edit'] == 'External censorship']
print(pd.to_numeric(df_cen['Eksamensgennemsnit']).describe(), '\n ', pd.to_numeric(df_nocen['Eksamensgennemsnit']).describe())


count    5144.000000
mean        6.634512
std         2.337830
min        -1.500000
25%         5.041054
50%         6.963850
75%         8.436648
max        12.000000
Name: Eksamensgennemsnit, dtype: float64 
  count    5536.000000
mean        7.189222
std         2.245170
min        -3.000000
25%         5.805926
50%         7.483249
75%         8.840227
max        12.000000
Name: Eksamensgennemsnit, dtype: float64


In [117]:
with open('df_final_outer-Copy1.pkl', 'rb') as f:
    dfo = pickle.load(f)

In [122]:
dfo[dfo['Year'] == '2013-2014']

,Kursus,Fakultet,Institut,Termin,ECTS,Eksamenstype,Antal tilmeldte,Fremmødte,Antal bestået,Eksamensgennemsnit,...,Colloquia_edit,Laboratory_edit,E-Learning_edit,Practical Training_edit,Praksishold_edit,Tutoring_edit,Manuduktion_edit,Language_edit,Type of assessmet_edit,URL_C
21,§3 i praksis - Naturbeskyttelsesloven,Det Natur- og Biovidenskabelige Fakultet,Institut for Geovidenskab og Naturforvaltning,s14/B5,5,Ordinær eksamen,12,12,12,9.16667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Danish,Portfolio,https://kurser.ku.dk/archive/2013-2014/course/...
27,1. årsprojekt,Det Sundhedsvidenskabelige Fakultet,Folkesundhedsvidenskab,s14,15,Ordinær eksamen,59,59,59,9.40678,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,1.årsprojekt,Det Humanistiske Fakultet,"Institut for Medier, Erkendelse og Formidling",s14,75,Ordinær eksamen,64,60,60,7.16667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,20. århundredes filosofi,Det Humanistiske Fakultet,"Institut for Medier, Erkendelse og Formidling",s14,15,Ordinær eksamen,69,67,59,6.08955,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,306. Masterprojekt,Det Humanistiske Fakultet,"Institut for Medier, Erkendelse og Formidling",s14,15,Ordinær eksamen,9,9,9,9.44444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,5. CCP1 Danish Culture,Det Humanistiske Fakultet,"Saxo-Instituttet - Arkæologi, Etnologi, Histor...",s14,75,Ordinær eksamen,97,97,89,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,6. CCP2 Danish Culture,Det Humanistiske Fakultet,"Saxo-Instituttet - Arkæologi, Etnologi, Histor...",s14,75,Ordinær eksamen,45,39,37,8.66667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,7. CCP3 Danish Culture,Det Humanistiske Fakultet,"Saxo-Instituttet - Arkæologi, Etnologi, Histor...",s14,15,Ordinær eksamen,11,10,10,10.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,8. CCP4A Intensive Course in Danish Culture,Det Humanistiske Fakultet,"Saxo-Instituttet - Arkæologi, Etnologi, Histor...",s14,15,Ordinær eksamen,37,31,27,7.58065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,English,Written not under invigilation,https://kurser.ku.dk/archive/2013-2014/course/...
44,8. CCP4C Intensive Course in Danish Culture,Det Humanistiske Fakultet,"Saxo-Instituttet - Arkæologi, Etnologi, Histor...",s14,15,Ordinær eksamen,45,44,44,9.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,English,Written not under invigilation,https://kurser.ku.dk/archive/2013-2014/course/...


In [161]:
dfo[dfo['URL_E'] == 'http://karakterstatistik.stads.ku.dk/Histogram/HEEK03471E/Summer-2018' ][]

,Kursus,Fakultet,Institut,Termin,ECTS,Eksamenstype,Antal tilmeldte,Fremmødte,Antal bestået,Eksamensgennemsnit,...,Colloquia_edit,Laboratory_edit,E-Learning_edit,Practical Training_edit,Praksishold_edit,Tutoring_edit,Manuduktion_edit,Language_edit,Type of assessmet_edit,URL_C
13307,Kultur og historie,Det Humanistiske Fakultet,"Saxo-Instituttet - Arkæologi, Etnologi, Histor...",s18,15,Ordinær eksamen,19,17,15,8.70588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
